In [ ]:
from tensorflow.keras import layers as L
from tensorflow.keras import Model
import  efficientnet.tfkeras as efn
import tensorflow as tf

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
def normalize(image):
  # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/main.py#L325-L326
  # https://github.com/tensorflow/tpu/blob/master/models/official/efficientnet/efficientnet_builder.py#L31-L32
  image -= tf.constant([0.485 * 255, 0.456 * 255, 0.406 * 255])  # RGB
  image /= tf.constant([0.229 * 255, 0.224 * 255, 0.225 * 255])  # RGB
  return image

def get_model(input_size, backbone='efficientnet-b5', weights='noisy-student', tta=False):
  print(f'Using backbone {backbone} and weights {weights}')
  x = L.Input(shape=input_size, name='imgs', dtype='float32')
  y = normalize(x)
  if backbone.startswith('efficientnet'):
    model_fn = getattr(efn, f'EfficientNetB{backbone[-1]}')

  y = model_fn(input_shape=input_size, weights=weights, include_top=False)(y)
  y = L.GlobalAveragePooling2D()(y)
  y = L.Dropout(0.2)(y)
  # 1292 of 1295 are present
  y = L.Dense(1292, activation='softmax')(y)
  model = tf.keras.Model(x, y)


  return model

In [ ]:
size=(160,256)

In [ ]:
model = get_model(input_size=size + (3, ),weights=None)
model.load_weights("model-B5-25-TPU.h5")

In [ ]:
model.save("B5-25.h5")

In [ ]:
model.summary()